In [1]:
import csv
import numpy as np
import tensorflow as tf

from itertools import islice

In [2]:
def load_data(file_name = 'train.csv' , one_hot = True):
    csv_reader = csv.reader(open(file_name))

    train_set = []
    count = 0
    train_labels = []
    for row in islice(csv_reader ,1 , None):
        row = list(map(np.float32 , row)) #类型转换
        train_set.append(row)

        temp = [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
        temp[int(train_set[count][0])] = 1.
        train_labels.append(temp)

        count += 1
    
    for i in range(count):
        train_set[i] = train_set[i][1:785]
    
    train_set = np.matrix(train_set)
    train_labels = np.matrix(train_labels)
    
    return train_set , train_labels

In [6]:
#加载数据
train_set , train_labels = load_data()

In [8]:
mini_batch = 0

def next_batch(train_set , train_labels , batch_size = 600):
    global mini_batch
    if mini_batch + batch_size > 40000:
        mini_batch = 0

    train_set_batch = train_set[mini_batch : mini_batch + batch_size]
    train_labels_batch = train_labels[mini_batch : mini_batch + batch_size]
    
    mini_batch = mini_batch + batch_size
    
    return train_set_batch , train_labels_batch

In [9]:
x = tf.placeholder(tf.float32 , shape = [None , 784])
y_ = tf.placeholder(tf.float32 , shape = [None , 10])

In [33]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape , stddev = 0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1 , shape = shape)
    return tf.Variable(initial)

def conv2d(x , W):
    return tf.nn.conv2d(x , W , strides=[1,1,1,1] , padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x , ksize = [1 , 2 , 2 , 1] , strides=[1,2,2,1] , padding='SAME')

x_image = tf.reshape(x , [-1 , 28,28,1])
#mean , variance = tf.nn.moments(x_image , [0,1,2])
#x_image = tf.nn.batch_normalization(x_image , mean=mean , variance=variance , offset=0 , scale=1 , variance_epsilon=1e-10)

#==============
#第一层卷积
#==============
W_conv1 = weight_variable([5,5,1,32])
b_conv1 = bias_variable([32])
h_conv1 = tf.nn.relu(conv2d(x_image , W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)
h_pool1 = tf.nn.lrn(h_pool1 , depth_radius=4 , bias= 1, alpha=0.0001 , beta=0.75)

#==============
#第二层卷积
#==============
W_conv2 = weight_variable([5,5,32,64])
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1 , W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)
h_pool2 = tf.nn.lrn(h_pool2 , depth_radius=4 , bias= 1, alpha=0.0001 , beta=0.75)
#==============
#全连接层（密集连接层）
#==============

W_fc1 = weight_variable([7*7*64 , 1024])
b_fc1 = bias_variable([1024])
h_pool2_flat = tf.reshape(h_pool2 , [-1 , 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat , W_fc1) + b_fc1)

#==============
#dropout
#==============
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1 , keep_prob)

#==============
#output layer softmax
#==============
W_fc2 = weight_variable([1024,10])
b_fc2 = bias_variable([10])
logits = tf.matmul(h_fc1_drop , W_fc2 ) + b_fc2

#==============
#cost function
#==============
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits , labels=y_)

#==============
#train
#==============
train_step = tf.train.AdamOptimizer(1e-3).minimize(cross_entropy)
#尝试 0.005 变慢了
sess = tf.Session()
sess.run(tf.global_variables_initializer())


dev_feed = {x:train_set[40000:] , y_:train_labels[40000:] , keep_prob:1.0}
right = tf.equal(tf.argmax(y_ , 1) , tf.argmax(tf.nn.softmax(logits) , 1))
accuracy = tf.reduce_mean(tf.cast(right , tf.float32))

for i in range(200):
    train_set_batch , train_labels_batch = next_batch(train_set , train_labels , batch_size = 100)
    sess.run(train_step , feed_dict = {x:train_set_batch , y_:train_labels_batch , keep_prob:0.8})
    
    if i % 10 == 0:
        print(sess.run(accuracy , feed_dict = dev_feed))


0.123
0.2925
0.577
0.7345
0.7575
0.8125
0.8375
0.8245
0.83
0.8295
0.8365
0.821
0.8205
0.8435
0.8305
0.8545
0.857
0.859
0.8565
0.873


In [13]:
def load_test_set(file_name = 'test.csv'):
    csv_reader = csv.reader(open(file_name))
    test_set = []
    
    for row in islice(csv_reader ,1 , None):
        row = list(map(np.float32 , row))
        test_set.append(row)
        
    test_set = np.matrix(test_set)

    return test_set

In [14]:
test_set = load_test_set()

In [30]:
mini_batch_test = 0

def read_next_batch(test_set , batch_size = 2800):
    global mini_batch_test
    
    test_set_batch = test_set[mini_batch_test : mini_batch_test + batch_size]
    
    mini_batch_test = mini_batch_test + batch_size
    
    return test_set_batch

In [31]:
csv_file = open('sample_submission.csv' , 'w' , newline='')
writer = csv.writer(csv_file)
writer.writerow(['ImageId','Label'])

15

In [32]:
test_set_labels = tf.argmax(tf.nn.softmax(logits) , 1)

k=1
for i in range(10):
    mini_test_set = read_next_batch(test_set)
    result = sess.run(test_set_labels , feed_dict={x:mini_test_set , keep_prob:1.0})
    
    for j in range(2800):
        w=j+k
        writer.writerow([w , result[j]])
    k+=2800

csv_file.close()